In [ ]:
!pip install tqdm
!pip install scikit-learn
!pip install torch
!pip install xgboost --no-cache-dir

In [ ]:
from pathlib import Path
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
# replace with own direction
LOCAL_REPOSITORY_DIR = "/Users/enfants/Code/OWKIN_ML2"


### Data Architecture


```
your_data_dir/
├── train_output.csv
├── train_input/
│   ├── images/
│       ├── ID_001/
│           ├── ID_001_tile_000_17_170_43.jpg
...
│   └── moco_features/
│       ├── ID_001.npy
...
├── test_input/
│   ├── images/
│       ├── ID_003/
│           ├── ID_003_tile_000_16_114_93.jpg
...
│   └── moco_features/
│       ├── ID_003.npy
...
├── supplementary_data/
│   ├── baseline.ipynb
│   ├── test_metadata.csv
│   └── train_metadata.csv
```

For instance, `your_data_dir = /storage/DATA_CHALLENGE_ENS_2022/`


## Data Loading

In [ ]:
# put your own path to the data root directory (see example in `Data architecture` section)
data_dir = Path(LOCAL_REPOSITORY_DIR)

# load the training and testing data sets
train_features_dir = data_dir / "train_input" / "moco_features"
test_features_dir = data_dir / "test_input" / "moco_features"
df_train = pd.read_csv(data_dir  / "supplementary_data" / "train_metadata.csv")
df_test = pd.read_csv(data_dir  / "supplementary_data" / "test_metadata.csv")

# concatenate y_train and df_train
y_train = pd.read_csv(data_dir  / "train_output.csv")
df_train = df_train.merge(y_train, on="Sample ID")

print(f"Training data dimensions: {df_train.shape}")  # (344, 4)
df_train.head()

## Data processing

In [ ]:
X_train = []
y_train = []
centers_train = []
patients_train = []
samples_train = []

for sample, label, center, patient in tqdm(  # loading bar
    df_train[["Sample ID", "Target", "Center ID", "Patient ID"]].values
):
    # load the coordinates and features (1000, 3+2048)
    _features = np.load(train_features_dir / sample)
    # split coords / features
    coordinates, features = _features[:, :3], _features[:, 3:]  # Ks

    # append each tile as a training sample
    X_train.append(features)
    y_train.append(np.full(len(features), label))
    centers_train.append(np.full(len(features), center))
    patients_train.append(np.full(len(features), patient))
    samples_train.append(np.full(len(features), sample))

# convert to numpy arrays
X_train = np.vstack(X_train)          # (N_tiles, 2048)
y_train = np.concatenate(y_train)     # (N_tiles,)
centers_train = np.concatenate(centers_train)
patients_train = np.concatenate(patients_train)
samples_train = np.concatenate(samples_train)

print(X_train.shape, y_train.shape)

## Cross Validation Setup

In [ ]:
# global configuration
CV_OUTER_SPLITS = 5
CV_OUTER_REPEATS = 2
CV_INNER_SPLITS = 3
MAX_TILES_PER_PATIENT = 150
AGG_CANDIDATES = ["mean", "median", "topk_mean"]
TOPK_FRAC = 0.10
ROBUST_SCORE_WEIGHTS = {"global_auc": 0.8, "center_min_auc": 0.2}
SELECTED_MODEL_NAME = "rf"  # options: "lr", "hgb", "rf"


def set_global_seed(seed):
    import random

    random.seed(seed)
    np.random.seed(seed)

    try:
        import torch

        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)
        if hasattr(torch.backends, "cudnn"):
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
    except Exception:
        # torch may be unavailable at this stage
        pass


set_global_seed(GLOBAL_SEED)


In [ ]:
# /!\ we perform splits at the patient level so that all samples from the same patient
# are found in the same split

patients_unique = np.unique(patients_train)
y_unique = np.array(
    [np.mean(y_train[patients_train == p]) for p in patients_unique]
)
centers_unique = np.array(
    [centers_train[patients_train == p][0] for p in patients_unique]
)

print(
    "Training set specifications\n"
    "---------------------------\n"
    f"{len(X_train)} unique samples\n"
    f"{len(patients_unique)} unique patients\n"
    f"{len(np.unique(centers_unique))} unique centers"
)

In [ ]:
# simple patient-level tile sampling (XX tiles / patient)

rng = np.random.RandomState(GLOBAL_SEED)

keep_idx = []
meta = []

for p in np.unique(patients_train):
    idx = np.where(patients_train == p)[0]

    if len(idx) > MAX_TILES_PER_PATIENT:
        idx = rng.choice(idx, MAX_TILES_PER_PATIENT, replace=False)

    keep_idx.append(idx)
    meta.append({
        "patient": p,
        "n_tiles_before": np.sum(patients_train == p),
        "n_tiles_after": len(idx),
        "label": y_train[idx][0],
    })

keep_idx = np.concatenate(keep_idx)

# apply sampling
X_train = X_train[keep_idx]
y_train = y_train[keep_idx]
centers_train = centers_train[keep_idx]
patients_train = patients_train[keep_idx]
samples_train = samples_train[keep_idx]


## CV Function

In [ ]:
def run_cv(model, X, y, patients, n_splits=5):
    
    patients_unique = np.unique(patients)
    y_unique = np.array([y[patients == p][0] for p in patients_unique])

    aucs = []

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    for fold, (train_pat_idx, val_pat_idx) in enumerate(
        skf.split(patients_unique, y_unique)
    ):
        train_patients = patients_unique[train_pat_idx]
        val_patients = patients_unique[val_pat_idx]

        train_idx = np.where(pd.Series(patients).isin(train_patients))[0]
        val_idx = np.where(pd.Series(patients).isin(val_patients))[0]

        X_train_fold, y_train_fold = X[train_idx], y[train_idx]
        X_val_fold, y_val_fold = X[val_idx], y[val_idx]

        model.fit(X_train_fold, y_train_fold)
        tile_preds = model.predict_proba(X_val_fold)[:, 1]

        df_val = pd.DataFrame({
            "patient": patients[val_idx],
            "pred": tile_preds,
            "y": y_val_fold
        })
        
        # agregation tile / patient 
        patient_pred = df_val.groupby("patient")["pred"].mean()
        patient_y = df_val.groupby("patient")["y"].first()

        auc = roc_auc_score(patient_y, patient_pred)
        aucs.append(auc)

        print(f"Fold {fold+1} AUC: {auc:.3f}")

    print(f"\nMean AUC: {np.mean(aucs):.3f}")
    return np.mean(aucs)

## 1. Baseline: Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(
    C=0.001,
    solver="liblinear",
    class_weight="balanced",
    random_state=GLOBAL_SEED,
)

mean_auc, std_auc = run_5fold_cv(lr, X_train, y_train,
                           patients_train,
                           patients_unique,
                           y_unique)


## 2. HistGradientBoostingClassifier

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(
    max_depth=6,
    learning_rate=0.05,
    max_iter=200,
    random_state=GLOBAL_SEED,
)


In [ ]:
mean_auc, std_auc = run_5fold_cv(hgb, X_train, y_train,
                           patients_train,
                           patients_unique,
                           y_unique)

## 3. Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=150,
    max_depth=12,
    min_samples_leaf=5,
    class_weight="balanced",
    n_jobs=-1,
    random_state=42
)

In [ ]:
run_cv(model, X_train, y_train, patients_train)

## + Multi-Layer Perceptron

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim=2048, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
device = torch.device("cpu")
print(device)

In [ ]:
EPOCHS = 5
BATCH_SIZE = 256

In [ ]:
def train_mlp(X_train, y_train, X_val, y_val,
              input_dim,
              hidden_dim=128,
              lr=1e-3,
              epochs=EPOCHS,
              batch_size=BATCH_SIZE):

    Xtr = torch.tensor(X_train, dtype=torch.float32).to(device)
    ytr = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)

    Xval = torch.tensor(X_val, dtype=torch.float32).to(device)
    yval = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1).to(device)

    model = MLP(input_dim=input_dim, hidden_dim=hidden_dim).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        perm = torch.randperm(Xtr.size(0))

        for i in range(0, Xtr.size(0), batch_size):
            idx = perm[i:i+batch_size]
            xb = Xtr[idx]
            yb = ytr[idx]

            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        val_logits = model(Xval)
        preds_val = torch.sigmoid(val_logits).cpu().numpy().ravel()

    return preds_val

In [ ]:
aucs = []
lrs = []

for k in range(5):
    kfold = StratifiedKFold(5, shuffle=True, random_state=k)
    fold = 0

    for train_idx_, val_idx_ in kfold.split(patients_unique, y_unique):

        # tile indexes for patients in fold
        train_idx = np.arange(len(X_train))[
            pd.Series(patients_train).isin(patients_unique[train_idx_])
        ]
        val_idx = np.arange(len(X_train))[
            pd.Series(patients_train).isin(patients_unique[val_idx_])
        ]

        # folds
        X_fold_train = X_train[train_idx]
        y_fold_train = y_train[train_idx]
        X_fold_val = X_train[val_idx]
        y_fold_val = y_train[val_idx]

        scaler = StandardScaler().fit(X_fold_train)
        X_fold_train = scaler.transform(X_fold_train)
        X_fold_val = scaler.transform(X_fold_val)

        preds_val = train_mlp(
            X_fold_train,
            y_fold_train,
            X_fold_val,
            y_fold_val,
            input_dim=X_fold_train.shape[1],
            hidden_dim=128,
            lr=1e-3,
            epochs=EPOCHS
        )
        
        # ---- PATIENT-LEVEL AGGREGATION ----
        val_pat = patients_train[val_idx]
        df_val = pd.DataFrame({
            "patient": val_pat,
            "pred": preds_val,
            "y": y_fold_val
        })

        patient_pred = df_val.groupby("patient")["pred"].mean()
        patient_y = df_val.groupby("patient")["y"].first()

        auc = roc_auc_score(patient_y, patient_pred)

        print(f"AUC on split {k} fold {fold}: {auc:.3f}")
        aucs.append(auc)
        lrs.append(lr)

        fold += 1

    print("----------------------------")

print(
    f"5-fold cross-validated PATIENT-level AUC averaged over {k+1} repeats: "
    f"{np.mean(aucs):.3f} ({np.std(aucs):.3f})"
)

# Submission

Now we evaluate the previous models trained through cross-validation so that to produce a submission file that can directly be uploaded on the data challenge platform.

## Data processing

In [ ]:
X_test_tiles = []
sample_ids_test_tiles = []

# keep test preprocessing at tile-level with deterministic cap
max_test_tiles_per_sample = MAX_TILES_PER_PATIENT if "MAX_TILES_PER_PATIENT" in globals() else None
rng_test = np.random.RandomState(GLOBAL_SEED)

# load the test data from `df_test` (~ 1 minute)
for sample in tqdm(df_test["Sample ID"].values):
    _features = np.load(test_features_dir / sample)
    features = _features[:, 3:]

    if max_test_tiles_per_sample is not None and len(features) > max_test_tiles_per_sample:
        keep = rng_test.choice(len(features), max_test_tiles_per_sample, replace=False)
        features = features[keep]

    X_test_tiles.append(features)
    sample_ids_test_tiles.append(np.full(len(features), sample))

X_test_tiles = np.vstack(X_test_tiles)
sample_ids_test_tiles = np.concatenate(sample_ids_test_tiles)


## Inference

In [ ]:
final_model = RandomForestClassifier(
    n_estimators=150,
    max_depth=12,
    min_samples_leaf=5,
    class_weight="balanced",
    n_jobs=-1,
    random_state=42
)

final_model.fit(X_train, y_train)

In [ ]:
preds_test = []

for sample in df_test["Sample ID"].values:
    _features = np.load(test_features_dir / sample)
    features = _features[:, 3:]

    tile_preds = final_model.predict_proba(features)[:, 1]
    slide_pred = np.mean(tile_preds)

    preds_test.append(slide_pred)

preds_test = np.array(preds_test)

## Saving predictions

In [ ]:
submission = pd.DataFrame(
    {"Sample ID": df_test["Sample ID"].values, "Target": preds_test}
).sort_values(
    "Sample ID"
)  # extra step to sort the sample IDs

# sanity checks
assert submission["Target"].notna().all(), "Some samples have missing predictions"
assert all(submission["Target"].between(0, 1)), "`Target` values must be in [0, 1]"
assert submission.shape == (len(df_test), 2), "Your submission file must match test size"
assert submission.shape == (149, 2), "Your submission file must be of shape (149, 2)"
assert list(submission.columns) == [
    "Sample ID",
    "Target",
], "Your submission file must have columns `Sample ID` and `Target`"

# save the submission as a csv file
submission.to_csv(data_dir / "output" / "Random_Forest_tile_level.csv", index=None)
submission.head()


# Dealing with images

The following code aims to load and manipulate the images provided as part of  this challenge.

## Scanning images paths on disk

This operation can take up to 5 minutes.

In [ ]:
train_images_dir = data_dir / "train_input" / "images"
train_images_files = list(train_images_dir.rglob("*.jpg"))

test_images_dir = data_dir / "test_input" / "images"
test_images_files = list(test_images_dir.rglob("*.jpg"))

print(
    f"Number of images\n"
    "-----------------\n"
    f"Train: {len(train_images_files)}\n" # 344 x 1000 = 344,000 tiles
    f"Test: {len(test_images_files)}\n"  # 149 x 1000 = 149,000 tiles
    f"Total: {len(train_images_files) + len(test_images_files)}\n"  # 493 x 1000 = 493,000 tiles
)

## Reading

Now we can load some of the `.jpg` images for a given sample, say `ID_001`.

In [ ]:
ID_001_tiles = [p for p in train_images_files if 'ID_001' in p.name]

In [ ]:
fig, axes = plt.subplots(5, 5)
fig.set_size_inches(12, 12)

for i, img_file in enumerate(ID_001_tiles[:25]):
    # get the metadata from the file path
    _, metadata = str(img_file).split("tile_")
    id_tile, level, x, y = metadata[:-4].split("_")
    img = plt.imread(img_file)
    ax = axes[i//5, i%5]
    ax.imshow(img)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(f"Tile {id_tile} ({x}, {y})")
plt.show()

## Mapping with features

Note that the coordinates in the features matrices and tiles number are aligned.

In [ ]:
sample = "ID_001.npy"
_features = np.load(train_features_dir / sample)
coordinates, features = _features[:, :3], _features[:, 3:]
print("xy features coordinates")
coordinates[:10, 1:].astype(int)

In [ ]:
print(
    "Tiles numbering and features coordinates\n"
)
[tile.name for tile in ID_001_tiles[:10]]